### Notebook Summary:
   - This notebook contains the code to merge historical stock data from Yahoo finance with Long and Short Ideas from Seeking Alpha
   - converting to datetime- both 
   - ideas- length of ticker, frequency of ticker, editors picks
   - offsets for different holding periods 

### Importing Necessary Libraries

In [11]:
import requests
import time
from bs4 import BeautifulSoup
import pandas as pd

In [12]:
final_dataframe= pd.read_csv('final_dataframe.csv')

In [13]:
long = final_dataframe[final_dataframe['Strategy']=='Long']
short = final_dataframe[final_dataframe['Strategy']=='Short']

long_tickers = long.drop_duplicates('Tickers')
short_tickers = short.drop_duplicates('Tickers')

long_links = long_tickers['Link'].tolist()
short_links = short_tickers['Link'].tolist()

language_links = long_links + short_links

language_dataframe = final_dataframe[final_dataframe['Link'].isin(language_links)]
language_dataframe.head(3)

,Authors,Link,Title,Strategy,Tickers,Date,Opening Price,Week1,Week2,Week3,...,Week44,Week45,Week46,Week47,Week48,Week49,Week50,Week51,Week52,Performance
0,Mark Hibben,/article/4034776-apple-oddly-satisfying-touchs...,Apple: The 'Oddly Satisfying' Touchscreen Mac,Long,AAPL,2017-01-03,116.150002,118.989998,120.000000,120.080002,...,174.250000,173.970001,169.979996,174.089996,169.800003,172.669998,176.419998,170.570007,116.150002,0
13,Michael Blair,/article/4037110-weak-demand-may-impact-apple-...,Weak Demand May Impact Apple Stock,Short,AAPL,2017-01-17,120.000000,120.080002,121.629997,130.289993,...,169.979996,174.089996,169.800003,172.669998,176.419998,170.570007,116.150002,118.989998,120.000000,0
282,Alex Cho,/article/4034871-amd-vs-intel-nvidia,AMD Vs. Intel And Nvidia,Long,AMD,2017-01-03,11.430000,11.490000,9.820000,9.910000,...,11.930000,11.090000,11.340000,11.550000,10.030000,10.160000,10.980000,10.460000,11.430000,0


In [10]:
language_dataframe['Performance'] = language_dataframe['Week12'] - language_dataframe['Opening Price']

language_dataframe['Performance'] = language_dataframe['Performance'].apply(lambda x: 0 if x > 0.0 else 1)

In [256]:
# Empty list to append results to
request_list = [] 
# List of links to iterate through

for link in links[2980:2988]: # Iterating through list of links to gather article text
    response = requests.get('https://seekingalpha.com'+(link), headers = {'user-agent': '-6787994961--capstone-'})
    # 'str(id_list)' is article link being scraped
    # Header included for polite scraping- way for website owners to contact you 
    time.sleep(2)
    print(response.status_code)
    html = response.text
    soup = BeautifulSoup(html,'lxml')
    request_list.append(soup) # Append results to empty list  
    print(link)

200
/article/4118215-exxon-mobil-strong-quarter-buoyed-higher-oil-prices
200
/article/4118192-exxon-mobil-oil-2-year-highs-stock-will-rise
200
/article/4118030-exxon-mobil-fully-valued-buy-tranches
200
/article/4121563-exxon-mobil-rating-update
200
/article/4127778-exxon-places-fiscal-sustainability-ahead-buybacks
200
/article/4133554-exxon-time-enjoy-acceleration
200
/article/4134317-2-things-will-drive-exxon-mobils-cash-flow-earnings-2018
200
/article/4134200-exxon-mobil-ready-break


## Engineer function to extract data

In [257]:
# Function to extract article text
def extract_article_text(request_list): 
    articles = []   # Empty list to append article text to 
    for i in request_list:  # Iterate through scraped results 
        for each in i.find_all('div',{'id':'a-body'}): # Call proper HTML tags/classes 
            articles.append(each.text)  # Append text data to empty list
    return(articles)

# Set function as variable
fetch_articles = extract_article_text(request_list)

## Combine results into DataFrames and Export

In [258]:
# Create dataframes with fetched results as column (total- 30 dataframes)
text = pd.DataFrame(fetch_articles)

In [259]:
text.to_csv('content176.csv')